In [ ]:
#머신러닝 2일차 과제_임소영
#white wine파일
#피쳐 값을 사용자가 입력했을 때 어떤 등급의 와인인지 나오게 끔 머신러닝을 하라

(1) 모듈 로딩 + 데이터 준비

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
file = '../data/winequality-white.csv'
data = pd.read_csv(file, sep=';')
data

In [ ]:
# 결측치 확인
data.isna().sum()

In [ ]:
# 형변환
data['quality'] = data['quality'].astype('float64')
data.info()

In [ ]:
data['quality'].value_counts()

In [ ]:
result = data.corr()
result

In [ ]:
result.loc['quality']

In [ ]:
# 상관계수를 뽑았을때 alcohol과 density가 가장 관련성이 높으므로 이 둘을 피쳐로 설정해보자
# 타겟 = quality
# 피쳐 = density, alcohol
file = '../data/winequality-white.csv'
wine = pd.read_csv(file, sep=';', usecols = ['density', 'alcohol', 'quality'])
wine

In [ ]:
plt.scatter(wine['density'], wine['alcohol'], alpha = 0.5)
plt.show()

(2) 데이터 전처리

In [ ]:
wine.shape

In [ ]:
# 중복값 확인
wine.duplicated().sum()

In [ ]:
wine.drop_duplicates(keep = 'first', inplace = True)
wine.shape

(3-1) 학습용 데이터 준비 => 피쳐와 타겟을 분리함  

In [ ]:
# 먼저 피쳐와 타겟을 분리함
feature = wine[['density', 'alcohol']]
feature

In [ ]:
target = wine['quality']
target

(3-2) 학습용 데이터 준비 => 학습 / 테스트 데이터 셋 분리

In [ ]:
# 학습/테스트 데이터셋을 분리시켜보자
# 학습 : 테스트 = 80 : 20

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size = 0.2, stratify = target)

In [ ]:
# 학습 및 테스트 데이터셋을 확인해보자
print('학습용 데이터셋')
print(f"X_train.shape: {X_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print()
print(y_train.value_counts()/y_train.shape[0])
print()
print('테스트용 데이터셋')
print(f"X_test.shape: {X_test.shape}")
print(f"y_test.shape: {y_test.shape}")

(3-3) 학습용 데이터 준비 => 피쳐 데이터의 스케일링

In [ ]:
# 피쳐들의 범위가 다르니까 그것을 통일시켜보자

from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler

In [ ]:
scaler.fit(X_train)

In [ ]:
#학습용 데이터 피처를 스케일링 해보자
scaled_X_train = scaler.transform(X_train)
scaled_X_train.shape

In [ ]:
# 테스트용 데이터 피처를 스케일링 해보자
scaled_X_test = scaler.transform(X_test)
scaled_X_test.shape

In [ ]:
# 시각화 해보자 - 원본
plt.scatter(X_train['density'], X_train['alcohol'], c=y_train)
plt.xlabel('density')
plt.ylabel('alcohol')
plt.title('BEFORE SCALE')
plt.show()

In [ ]:
scaled_X_train

In [ ]:
scaled_X_train[:,0].max()
scaled_X_train[:,0].min()

In [ ]:
# 시각화 해보자 - 스케일링 후
plt.scatter(scaled_X_train[:,0], scaled_X_train[:,1], c = y_train)
plt.xlabel('density')
plt.ylabel('alcohol')
plt.title('AFTER SCALE')
plt.show()

(4) 학습 진행

In [ ]:
model = KNeighborsClassifier()
model

In [ ]:
# 학습
model.fit(scaled_X_train, y_train)

In [ ]:
# 테스트
model.score(scaled_X_test, y_test)

In [ ]:
# 예측
user_data = input('와인 품종을 확인하기 위한 데이터를 입력하세요 (density, alcohol 순서로 입력: ')
user_list = user_data.split(' ')
user_density = float(user_list[0])
user_alcohol = float(user_list[1])

In [ ]:
new_data = pd.DataFrame([[user_density, user_alcohol]], columns = ['density', 'alcohol'])
new_data

In [ ]:
# 피쳐를 스케일링 시켜보자
scaled_new_data = scaler.transform(new_data)

In [ ]:
y_pre = model.predict(scaled_new_data)
y_pre

In [ ]:
# 방법 : KNN 알고리즘
# max_k = scaled_X_train.shape[0]
# max_k

In [ ]:
'''
train_scoreList = []
test_scoreList = []
for k in range(1, max_k + 1):
    model = KNeighborsClassifier(n_neighbors=k)
    # 학습 진행
    model.fit(scaled_X_train, y_train)
    # 점수 확인
    train_scoreList.append(model.score(scaled_X_train, y_train))
    test_scoreList.append(model.score(scaled_X_test, y_test))
    
'''

In [ ]:
# 학습 및 테스트 데이터에 대한 점수 확인
#print(f"Train Score : {train_scoreList}")
#print(f"Test Score : {test_scoreList}")

In [ ]:
# K 개수에 따른 학습용/테스트용 점수 시각화
# plt.plot(train_scoreList, 'b--', label = 'Train Score')
# plt.plot(test_scoreList, 'r--', label = 'Test Score')
# plt.title('K & Score')
# plt.xlabel('K')
# plt.ylabel('Score')
# plt.legend()
# plt.show()